In [ ]:
# 퍼스널 컬러 학습 

In [ ]:
#데이터 증강
from imblearn.over_sampling import SMOTE

# SMOTE 적용 (소수 클래스 증강)
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# 증강 후 클래스 분포 확인
print(pd.Series(y_res).value_counts())

In [ ]:
# 특징 선택
from sklearn.feature_selection import SelectKBest, f_classif

# 상위 10개 특징 선택
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X_res, y_res)


In [ ]:
# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 최적 파라미터 탐색
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'class_weight': ['balanced', None]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_new, y_res)
print("Best Params:", grid_search.best_params_)

In [ ]:
# 교차 검증
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# 계층별 5-fold 교차 검증
cv = StratifiedKFold(n_splits=5)
for train_idx, test_idx in cv.split(X_new, y_res):
    X_train, X_test = X_new[train_idx], X_new[test_idx]
    y_train, y_test = y_res[train_idx], y_res[test_idx]
    
    clf = RandomForestClassifier(**grid_search.best_params_)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    print("F1-Score:", f1_score(y_test, preds, average='weighted'))


In [ ]:
# 대체 모델 실험
from xgboost import XGBClassifier
from sklearn.svm import SVC

# XGBoost 실험
xgb = XGBClassifier()
xgb.fit(X_new, y_res)

# SVM 실험 (스케일링 필수)
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X_new)
svm = SVC(kernel='rbf', class_weight='balanced')
svm.fit(X_scaled, y_res)


In [ ]:
# 최종 파이프라인
from sklearn.pipeline import Pipeline

final_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(f_classif, k=10)),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1
    ))
])

# 최종 평가
final_pipeline.fit(X_res, y_res)
print("Final Accuracy:", final_pipeline.score(X_test, y_test))
